# Mount Collab and Get Parquet Files

In [ ]:
!curl ipinfo.io

{
  "ip": "34.45.78.225",
  "hostname": "225.78.45.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
!ls -lh

total 2.5M
drwx------ 5 root root 4.0K Nov 14 21:15 drive
-rw-r--r-- 1 root root 2.5M Nov 14 21:15 eth_news_data.parquet.gzip
drwxr-xr-x 1 root root 4.0K Nov 12 14:30 sample_data


In [ ]:
!mv /content/*.parquet.gzip "/content/drive/MyDrive/KFUPM Study Material/Term 3 (251 - 2025)/ICS-590/crypto-forecasting-public/big-datasets"

# Google News Data Fetch

## Functions

In [ ]:
''' Functions for Google News data fetching. '''

import sys
sys.path.append('../../')

import re
import string
import random
import requests
import pandas as pd

from datetime import datetime, timedelta
from functools import lru_cache
from bs4 import BeautifulSoup, XMLParsedAsHTMLWarning
from email.utils import parsedate_to_datetime
import warnings

# Quiet the BS4 XML warning (we are explicitly parsing as XML)
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

# --- HTTP defaults (helps avoid sporadic empty feeds / blocks)
_REQ_TIMEOUT = 20
_REQ_HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36"
}


class GoogleNewsRSS:
    ''' Scraper for Google News RSS. '''

    def __init__(self, rss_url: str):
        self.url = rss_url
        try:
            self.response = requests.get(rss_url, headers=_REQ_HEADERS, timeout=_REQ_TIMEOUT)
            self.response.raise_for_status()
        except Exception as e:
            # Hard fail here; caller will handle
            raise RuntimeError(f"Request failed for Google News RSS: {rss_url} :: {e}")

        # Parse as XML (not HTML)
        try:
            self.soup = BeautifulSoup(self.response.text, features="xml")
        except Exception as e:
            raise RuntimeError(f"Could not parse XML from: {rss_url} :: {e}")

        # Always define self.articles
        channel = self.soup.find('channel')
        self.articles = [] if channel is None else channel.find_all('item')

        # Normalize per-item fields (be robust to tag name differences)
        def _get_text(tag):
            return tag.get_text(strip=True) if tag else None

        def _pubdate(item):
            # Google uses <pubDate>; be tolerant of case variants
            tag = item.find('pubDate') or item.find('pubdate')
            return _get_text(tag)

        self.articles_dicts = [{
            'title': _get_text(item.find('title')),
            'link': _get_text(item.find('link')),
            'description': _get_text(item.find('description')),
            'pubdate': _pubdate(item),
        } for item in self.articles]

        self.size = len(self.articles)
        self.urls = [d['link'] for d in self.articles_dicts if d.get('link')]
        self.titles = [d['title'] for d in self.articles_dicts if d.get('title')]
        self.descriptions = [d['description'] for d in self.articles_dicts if d.get('description')]
        self.publication_times = [d['pubdate'] for d in self.articles_dicts if d.get('pubdate')]


@lru_cache
def convert_time(time_str: str) -> datetime:
    """
    Google News RSS uses RFC2822 dates, e.g.:
    'Tue, 04 Mar 2025 12:34:56 GMT'
    Use email.utils.parsedate_to_datetime for reliability.
    """
    if not time_str:
        raise ValueError("Empty pubdate string")
    try:
        return parsedate_to_datetime(time_str)
    except Exception:
        # Fallback to the original regex-based approach if needed
        _format = '%d %b %Y %H:%M:%S'
        cleaned = re.sub(r'^.*?,', ',', time_str)[2:][:-4]
        return datetime.strptime(cleaned, _format)


def get_data(coin: str = 'BTC') -> pd.DataFrame:
    """
    Build a date-bounded Google News RSS query for BTC/ETH and
    aggregate daily results into a single DataFrame indexed by timestamp.
    """
    link_tmpl = string.Template(
        'https://news.google.com/rss/search?'
        'q=CoinDesk+OR+Cointelegraph+OR+Decrypt,+$currency+OR+$symbol+'
        'after:$early_date+before:$late_date&ceid=US:en&hl=en-US&gl=US'
    )

    if coin == 'BTC':
        currency = 'Bitcoin'
    elif coin == 'ETH':
        currency = 'Ethereum'
    else:
        raise ValueError(f'Coin not supported: {coin}')

    all_data = pd.DataFrame()

    # Start date as in your current code for BTC
    # c_date = datetime.strptime('01-01-2009', '%d-%m-%Y')
    # Start date as in your current code for ETH
    c_date = datetime.strptime('01-01-2013', '%d-%m-%Y')

    # Shuffle daily windows to reduce rate limits
    iterator = []
    while c_date <= datetime.now():
        iterator.append(c_date)
        c_date += timedelta(days=1)
    random.shuffle(iterator)

    for date in iterator:
        next_date = date + timedelta(days=1)
        url = link_tmpl.substitute(
            currency=currency,
            symbol=coin,
            early_date=date.strftime('%Y-%m-%d'),
            late_date=next_date.strftime('%Y-%m-%d'),
        )

        # req = GoogleNewsRSS(url)
        try:
            req = GoogleNewsRSS(url)
        except RuntimeError as e:
            # If it's a 503 or 429, log and skip that day; otherwise re-raise
            msg = str(e)
            if '503' in msg or '429' in msg:
                print(f"Transient error for {coin} {date.date()}: {msg} – skipping this day.")
                continue
            else:
                raise RuntimeError(f"Request failed for Google News RSS due to {e}")

        # No items for that window → skip
        if not req.publication_times:
            continue

        rows = list(zip(req.publication_times, req.titles, req.urls))
        c_df = pd.DataFrame(rows, columns=['time', 'title', 'url'])

        # Parse timestamps
        c_df['datetime'] = [convert_time(t) for t in c_df.time]
        c_df['timestamp'] = [dt.timestamp() for dt in c_df.datetime]
        c_df = c_df.drop(columns='time').set_index('timestamp')

        all_data = pd.concat([all_data, c_df], axis=0, ignore_index=False)

    print(f'All elements fetched. ({len(iterator)}/{len(iterator)})')
    if all_data.empty:
        # Make the failure explicit to the caller/runner
        raise RuntimeError("Google News RSS returned no rows for the chosen date range.")
    return all_data.sort_index().drop_duplicates()

print("Google News functions loaded.")


Google News functions loaded.


## Main Call

In [ ]:
# try:
#     btc_news_data = get_data('BTC')
#     btc_news_data.to_parquet('btc_news_data.parquet.gzip', compression='gzip')
#     print('Bitcoin news data successfully scraped \U0001F389 \U0001F389')
# except Exception as e:
#     print('Error raised during bitcoin news data scraping: ', e)

try:
    eth_news_data = get_data('ETH')
    eth_news_data.to_parquet('eth_news_data.parquet.gzip', compression='gzip')
    print('Ethereum news data successfully scraped \U0001F389 \U0001F389')
except Exception as e:
    print('Error raised during ethereum news data scraping: ', e)


All elements fetched. (6162/6162)
Bitcoin news data successfully scraped 🎉 🎉
Error raised during ethereum news data scraping:  Request failed for Google News RSS: https://news.google.com/rss/search?q=CoinDesk+OR+Cointelegraph+OR+Decrypt,+Ethereum+OR+ETH+after:2012-03-14+before:2012-03-15&ceid=US:en&hl=en-US&gl=US :: 503 Server Error: Service Unavailable for url: https://news.google.com/rss/search?q=CoinDesk+OR+Cointelegraph+OR+Decrypt,+Ethereum+OR+ETH+after:2012-03-14+before:2012-03-15&ceid=US:en&hl=en-US&gl=US


In [ ]:
try:
    eth_news_data = get_data('ETH')
    eth_news_data.to_parquet('eth_news_data.parquet.gzip', compression='gzip')
    print('Ethereum news data successfully scraped \U0001F389 \U0001F389')
except Exception as e:
    print('Error raised during ethereum news data scraping: ', e)

All elements fetched. (4701/4701)
Ethereum news data successfully scraped 🎉 🎉


# Google Trends Data

## Functions

In [ ]:
explore_url = "https://trends.google.com/trends/api/explore"
widget_url = 'https://trends.google.com/trends/api/widgetdata/multiline/csv'

cookies = {  # TODODEV: Replace with fresh cookies if requests fail
    'AEC': 'ARSKqsJPmyKVB...', # Placeholder for a real cookie value
}
headers = {  # TODODEV: Replace with fresh headers if requests fail
    'Host': 'trends.google.com',
    'Cookie': 'SID=g.a0002wiTNNCLxuBBFp3bme_jNqAuarl-iXHSwqbOHgUi5nDOVzibEDsiE1j66S2J90FQzYk6JgACgYKAYMSARYSFQHGX2MiRvI4RyQjCEumvcNJnDdTQRoVAUF8yKpQ55Gi3zTLn2wlWBOtf3Rv0076; __Secure-1PSID=g.a0002wiTNNCLxuBBFp3bme_jNqAuarl-iXHSwqbOHgUi5nDOVzibRMncR1cDVMHs9Ztdp8HqDgACgYKAbsSARYSFQHGX2MiQAuYYNmyAgCA8ZIgpAZBkBoVAUF8yKriDy_SPAkxplM9F0RriJJ20076; __Secure-3PSID=g.a0002wiTNNCLxuBBFp3bme_jNqAuarl-iXHSwqbOHgUi5nDOVzibVEWjkPP3f6K2TTgLL7yAxAACgYKAbYSARYSFQHGX2Mi45UbnvYsyp2ZYeuB5KaHJxoVAUF8yKqhubT8FrXSC51W2HCtFRtq0076; HSID=Az5DDIxBoZRWcNygu; SSID=AWGK1Y7qTITh0fQ-W; APISID=R6iPJ3t61HiJE2y3/AvIfC_e82kTUWE2wE; SAPISID=fBGtP9UIP8jh8nO5/A7lfg7aVR3K3BshN3; __Secure-1PAPISID=fBGtP9UIP8jh8nO5/A7lfg7aVR3K3BshN3; __Secure-3PAPISID=fBGtP9UIP8jh8nO5/A7lfg7aVR3K3BshN3; SEARCH_SAMESITE=CgQIoZ8B; AEC=AaJma5v35HSEDNr_n11dzInL9nRcMbgD4QOpcwFNMulHvR77GgQUaLfM8A; __Secure-STRP=ADq1D7ooCPPAX7__1oiug9RgTrbde_TRwdXpEE3GYo27v6xs9jiaOLA5isq2D_6-Bo9t1_KZIv4WvObeEqQua_9Q3pHcCd3JJVkP; __Secure-1PSIDTS=sidts-CjEBwQ9iI82vc7lZm3Mgcmuxolge6fRezWJx1jM7UHD6BCe9DihgtdLxQw26wOI6Nh1gEAA; __Secure-3PSIDTS=sidts-CjEBwQ9iI82vc7lZm3Mgcmuxolge6fRezWJx1jM7UHD6BCe9DihgtdLxQw26wOI6Nh1gEAA; NID=526=flH9BCywvl1cP6Rujl1g8zD0cgRcmgt8IirrE1Ae_fA1fYd9vPD1IgYODkDc5sMD8RN_oqBs8LWKg1zb0eq9Ux8ZGP1iWQ-aJRtv3-wu0HK6ATJrotEuvbWSETlfLo3eFiTFFo5R5VNDwjpLIkaTn4B9t1lz64bUfWONyJWMZ0v_s4Xqt-gjXtcu0hJd1giiTtgiod5EOZ7oF0siw6_AqV0Yij9wyHJspCHOPVDwKKREyQyaQZyuW5NbKfJD-teldbFjQbBHg03wFlbZLsFNEcdrtL0ik9zsT3UVuFXp8dFIPJS7Pxpz_5svTaXPDwn5XE2aUO_kwcBraOvsE5NSJ26lrFjVKo7tVZHYUDLa1YJFvPmE2xm7b-0QN7s; SIDCC=AKEyXzWvCKIhLI0dLYGcdPiLt0YMjOVjKiZwZNyMESYYF_N9xv_dQHs1rut-c5jI1LAyl-S-7w; __Secure-1PSIDCC=AKEyXzVt18wgIHH2vTQvk8FUC_H5_FzSSu484o4attajftgAn09Zh9Qzdi_0T3iJeZv-AUFemuI; __Secure-3PSIDCC=AKEyXzXi6WgXABCYvMN8nFCHMmG39i3gcoe8j_3UY97W_AHM_usPR6U6VzUEwcEc4mM7SJtjEmk',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
}
print("Google Trends config loaded (update TODODEV cookies/headers if needed).")


import sys
sys.path.append('../../')

import time, json, urllib, requests, pandas as pd
from io import StringIO
from datetime import datetime, timedelta

def fetch_explore_api(hl, timezone, keyword, from_date, to_date):
    # Ensure keyword is URL-encoded for safety
    encoded_keyword = urllib.parse.quote(str(keyword))
    # 'time' parameter in req should use 'YYYY-MM-DD YYYY-MM-DD' format
    time_param = f"{datetime.strftime(from_date,'%Y-%m-%d')} {datetime.strftime(to_date,'%Y-%m-%d')}"
    req_payload = {
        "comparisonItem":[{"keyword": encoded_keyword,"geo":"","time": time_param}],
        "category":0,
        "property":""
    }
    query = urllib.parse.urlencode([
        ("hl", hl),
        ("tz", timezone),
        ("req", json.dumps(req_payload))
    ])
    response = requests.get(explore_url + '?' + query, cookies=cookies, headers=headers)
    while response.status_code == 429:
        time.sleep(2) # Wait and retry for rate limits
        response = requests.get(explore_url + '?' + query, cookies=cookies, headers=headers)
    if response.status_code != 200:
        raise Exception(f'Explore API returned error {response.status_code}: {response.text}')
    # The JSON response starts with ')]}' + a newline, so skip that.
    return json.loads(response.text[5:])['widgets']

def fetch_widget_api(params: dict):
    # widget_url is already defined globally
    response = requests.get(widget_url, params=params, cookies=cookies, headers=headers)
    while response.status_code == 429:
        time.sleep(2) # Wait and retry for rate limits
        response = requests.get(widget_url, params=params, cookies=cookies, headers=headers)
    if response.status_code != 200:
        raise Exception(f'Widget API returned error {response.status_code}: {response.text}')
    return response # Return the response object, content will be accessed in get_trends_data

def get_trends_data(keywords: list, begin_date: datetime=None, timezone: str=str(-60), hl: str='en'):
    if begin_date is None:
        begin_date = datetime.strptime('2009-01-01', '%Y-%m-%d').date()
    end_date = datetime.now().date()
    all_data = pd.DataFrame()

    for keyword in keywords:
        current_to_date = end_date
        current_from_date = end_date - timedelta(days=250)

        keyword_dfs = [] # Collect dataframes for the current keyword

        # Loop to fetch data in 250-day windows backwards in time
        while True:
            # Ensure from_date doesn't go before begin_date
            if current_from_date < begin_date:
                current_from_date = begin_date

            # If current_to_date is before current_from_date, we're done or there's an issue
            if current_to_date < current_from_date:
                break

            # Handle cases where current_from_date and current_to_date are the same or to_date is earlier than from_date
            if current_from_date > current_to_date:
                break

            try:
                widgets = fetch_explore_api(hl, timezone, keyword, current_from_date, current_to_date)
            except Exception as e:
                print(f"Error fetching explore API for keyword '{keyword}' ({current_from_date} to {current_to_date}): {e}")
                # Adjust dates to try the next window
                if current_from_date == begin_date: # Cannot go further back
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue


            if not widgets: # Handle case where widgets might be empty
                print(f"No widgets returned for keyword '{keyword}' for period {current_from_date} to {current_to_date}. Skipping.")
                if current_from_date == begin_date:
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue

            token = widgets[0]['token']
            request = widgets[0]['request']
            params = {'req': json.dumps(request),'token': token,'tz': timezone}

            try:
                response = fetch_widget_api(params)
            except Exception as e:
                print(f"Error fetching widget API for keyword '{keyword}' ({current_from_date} to {current_to_date}): {e}")
                if current_from_date == begin_date:
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue

            csv_content = response.text

            # Find the actual CSV header (e.g., "Day","bitcoin")
            csv_lines = csv_content.strip().split('\n')

            header_row_index = -1
            for i, line in enumerate(csv_lines):
                # Look for lines that likely contain header for date and keyword data
                if ('"Day"' in line or 'Day' in line) and ('"' + keyword.lower() + '"' in line.lower() or keyword.lower() in line.lower() or 'Interest' in line):
                    header_row_index = i
                    break
                # A more generic check if keyword not explicitly in header line (e.g., "Day","Interest")
                elif ('"Day"' in line or 'Day' in line) and len(line.split(',')) > 1: # check for at least two columns
                    header_row_index = i
                    break

            if header_row_index == -1:
                print(f"Warning: Could not find 'Day' header for keyword '{keyword}' for period {current_from_date} to {current_to_date}. Skipping this window.")
                # This could happen for very old dates with no data.
                if current_from_date == begin_date:
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue

            try:
                # Read CSV, setting the first column ('Day' or date-like) as index and parsing as dates
                df = pd.read_csv(StringIO('\n'.join(csv_lines[header_row_index:])), index_col=0, parse_dates=True)
            except Exception as e:
                print(f"Error parsing CSV for keyword '{keyword}' ({current_from_date} to {current_to_date}): {e}. CSV content sample: {csv_content[:500]} Skipping this window.")
                if current_from_date == begin_date:
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue

            # Google Trends sometimes returns a 'isPartial' column.
            # We are interested in the main trend data.
            if 'isPartial' in df.columns:
                df = df.drop(columns=['isPartial'])

            if len(df.columns) > 1:
                # If there are still multiple columns (e.g., if the keyword itself is multi-word or ambiguous in GTrends),
                # try to pick the column that matches the keyword or is named 'Interest'.
                # Otherwise, default to the first column.
                if keyword in df.columns:
                    df = df[[keyword]]
                elif 'Interest' in df.columns: # Common for single keywords
                    df = df[['Interest']]
                else:
                    df = df.iloc[:, 0:1] # Fallback: take the first data column
            elif len(df.columns) == 0:
                print(f"Warning: DataFrame for keyword '{keyword}' for period {current_from_date} to {current_to_date} is empty after processing. Skipping.")
                if current_from_date == begin_date:
                    break
                current_to_date = current_from_date - timedelta(days=1)
                current_from_date = current_to_date - timedelta(days=250)
                continue

            df.columns = [keyword] # Rename the single data column to the actual keyword
            keyword_dfs.append(df)

            # Prepare for the next window
            current_to_date = current_from_date - timedelta(days=1)
            next_from_date_candidate = current_to_date - timedelta(days=250)

            if next_from_date_candidate < begin_date:
                current_from_date = begin_date
            else:
                current_from_date = next_from_date_candidate

            if current_to_date < begin_date: # If the 'to' date for the next window is before the overall begin_date, we're done
                break
            # If current_from_date has reached begin_date and we've processed up to current_to_date (which is now begin_date - 1),
            # we should break to prevent an infinite loop.
            if current_from_date == begin_date and current_to_date < begin_date:
                break


        # After collecting all windows for a keyword, concatenate them
        if keyword_dfs:
            complete_kw_data = pd.concat(keyword_dfs)
            # Drop duplicates by index (date), keeping the last value (most recent fetch for overlapping periods)
            complete_kw_data = complete_kw_data[~complete_kw_data.index.duplicated(keep='last')]
            complete_kw_data = complete_kw_data.sort_index()

            if all_data.empty:
                all_data = complete_kw_data
            else:
                # Use merge for column-wise concatenation with automatic index alignment
                all_data = pd.merge(all_data, complete_kw_data, left_index=True, right_index=True, how='outer')
        else:
            print(f"No data fetched for keyword: {keyword}")

    return all_data.sort_index()

print("Google Trends functions loaded.")

Google Trends config loaded (update TODODEV cookies/headers if needed).
Google Trends functions loaded.


## Main Call

In [ ]:
trends_data = get_trends_data(['bitcoin','ethereum','cryptocurrency','blockchain','investing'], timezone=str(0))
trends_data.to_parquet('google_trends.parquet.gzip', compression='gzip')

/tmp/ipython-input-4282577138.py:197: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  complete_kw_data = pd.concat(keyword_dfs)
/tmp/ipython-input-4282577138.py:197: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  complete_kw_data = pd.concat(keyword_dfs)
/tmp/ipython-input-4282577138.py:197: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  comp

# Yahoo Finance Data Fetch

## Functions and Call

In [ ]:
import pandas as pd
import yfinance as yf

start_date='2009-01-01'
end_date='2025-10-31'

sp500_data = yf.download("^GSPC", start=start_date, end=end_date)
vix_data   = yf.download("^VIX",  start=start_date, end=end_date)
gold_data  = yf.download("GC=F",  start=start_date, end=end_date)

# Safely pick Adj Close if available, otherwise Close
sp500_close = sp500_data.get('Adj Close', sp500_data['Close'])
vix_close   = vix_data.get('Adj Close', vix_data['Close'])
gold_close  = gold_data.get('Adj Close', gold_data['Close'])

yf_data = pd.concat([sp500_close, sp500_data['Volume'],
                     vix_close, gold_close], axis=1)

yf_data.columns = ['sp500_price', 'sp500_volume', 'vix_price', 'gold_usd_price']

yf_data.to_parquet('yf_data.parquet.gzip', compression='gzip')

print("✅ Yahoo Finance data saved to yf_data.parquet.gzip")


/tmp/ipython-input-271872396.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500_data = yf.download("^GSPC", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-271872396.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix_data   = yf.download("^VIX",  start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-271872396.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gold_data  = yf.download("GC=F",  start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

✅ Yahoo Finance data saved to yf_data.parquet.gzip


# Converting BTC and ETH CSV to Parquet

In [2]:
%cd /content/drive/MyDrive/KFUPM Study Material/Term 3 (251 - 2025)/ICS-590/crypto-forecasting-public/big-datasets

/content/drive/MyDrive/KFUPM Study Material/Term 3 (251 - 2025)/ICS-590/crypto-forecasting-public/big-datasets


In [3]:
!ls -lh

total 12M
-rw------- 1 root root 463K Nov 14 21:43  bitcoin_2010-07-17_2024-06-28.csv
-rw------- 1 root root 293K Nov 14 23:50  btc_data.parquet
-rw------- 1 root root 3.5M Nov 14 20:01  btc_news_data.parquet.gzip
-rw------- 1 root root  38K Nov 24 11:21 'Data Acquisition'
-rw------- 1 root root 219K Nov 14 23:50  eth_data.parquet
-rw------- 1 root root 339K Nov 14 21:43  ethereum_2015-08-07_2024-06-28.csv
-rw------- 1 root root 2.5M Nov 14 21:15  eth_news_data.parquet.gzip
-rw------- 1 root root 3.6M Nov 19 14:46  Fungineering.ipynb
-rw------- 1 root root  68K Nov 14 20:22  google_trends.parquet.gzip
-rw------- 1 root root  37K Nov 18 22:48  histories_classification_before_stopping.pkl
-rw------- 1 root root  16K Nov 19 13:25  histories_classification.pkl
-rw------- 1 root root 8.5K Nov 19 14:46  histories_regression_modified_new.pkl
-rw------- 1 root root 8.5K Nov 18 23:21  histories_regression_modified.pkl
-rw------- 1 root root 4.4K Nov 19 13:29  histories_regression_new.pkl
-rw---

In [ ]:
import pandas as pd

# Define input and output file paths
csv_file_path = 'bitcoin_2010-07-17_2024-06-28.csv'
parquet_file_path = 'btc_data.parquet'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Write the DataFrame to a Parquet file
df.to_parquet(parquet_file_path, index=False) # index=False prevents writing the DataFrame index as a column
print(f"CSV file '{csv_file_path}' converted to Parquet file '{parquet_file_path}' successfully.")

# Define input and output file paths
csv_file_path = 'ethereum_2015-08-07_2024-06-28.csv'
parquet_file_path = 'eth_data.parquet'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Write the DataFrame to a Parquet file
df.to_parquet(parquet_file_path, index=False) # index=False prevents writing the DataFrame index as a column
print(f"CSV file '{csv_file_path}' converted to Parquet file '{parquet_file_path}' successfully.")

CSV file 'bitcoin_2010-07-17_2024-06-28.csv' converted to Parquet file 'btc_data.parquet' successfully.
CSV file 'ethereum_2015-08-07_2024-06-28.csv' converted to Parquet file 'eth_data.parquet' successfully.


In [ ]:
import pandas as pd

# Specify the path to your Parquet file
btc_parquet_file_path = 'btc_data.parquet'
eth_parquet_file_path = 'eth_data.parquet'
btc_news_data_parquet_file_path = 'btc_news_data.parquet.gzip'
eth_news_data_parquet_file_path = 'eth_news_data.parquet.gzip'
google_trends_parquet_file_path = 'google_trends.parquet.gzip'
yf_data_parquet_file_path = 'yf_data.parquet.gzip'

df = pd.read_parquet(btc_parquet_file_path)
print("BTC DATA")
print(df.columns)
print(df.head(1))

df = pd.read_parquet(eth_parquet_file_path)
print("\nETH DATA")
print(df.columns)
print(df.head(1))

df = pd.read_parquet(btc_news_data_parquet_file_path)
print("\nBTC NEWS DATA")
print(df.columns)
print(df.head(1))

df = pd.read_parquet(eth_news_data_parquet_file_path)
print("\nETH NEWS DATA")
print(df.columns)
print(df.head(1))

df = pd.read_parquet(google_trends_parquet_file_path)
print("\nGOOGLE TRENDS DATA")
print(df.columns)
print(df.head(1))

df = pd.read_parquet(yf_data_parquet_file_path)
print("\nYAHOO FINANCE DATA")
print(df.columns)
print(df.head(1))



BTC DATA
Index(['Start', 'End', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')
        Start         End     Open      High       Low     Close  \
0  2024-06-27  2024-06-28  60882.9  62338.43  60666.19  61646.05   

         Volume    Market Cap  
0  1.215887e+11  1.208392e+12  

ETH DATA
Index(['Start', 'End', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')
        Start         End       Open       High        Low      Close  \
0  2024-06-27  2024-06-28  3383.5292  3481.1707  3379.1649  3458.8487   

         Volume    Market Cap  
0  1.580400e+10  4.116605e+11  

BTC NEWS DATA
Index(['title', 'url', 'datetime'], dtype='object')
                                          title  \
timestamp                                         
1.365404e+09  How To Mine Bitcoins - TechCrunch   

                                                            url  \
timestamp                                                         
1.365404e+09  https://

In [6]:
import pandas as pd
import os

# Define the directory containing your Parquet files
parquet_dir = './'
# Define the directory where you want to save the CSV files
csv_dir = 'csv_files'

# Create the output directory if it doesn't exist
os.makedirs(csv_dir, exist_ok=True)

# Iterate through all files in the Parquet directory
for filename in os.listdir(parquet_dir):
    if filename.endswith('.parquet.gzip'):
        parquet_filepath = os.path.join(parquet_dir, filename)

        # Read the Parquet file into a Pandas DataFrame
        df = pd.read_parquet(parquet_filepath)

        # Construct the output CSV filename
        csv_filename = filename.replace('.parquet.gzip', '.csv')
        csv_filepath = os.path.join(csv_dir, csv_filename)

        # Write the DataFrame to a CSV file (without index)
        df.to_csv(csv_filepath, index=False)
        print(f"Converted '{filename}' to '{csv_filename}'")

Converted 'btc_news_data.parquet.gzip' to 'btc_news_data.csv'
Converted 'google_trends.parquet.gzip' to 'google_trends.csv'
Converted 'yf_data.parquet.gzip' to 'yf_data.csv'
Converted 'eth_news_data.parquet.gzip' to 'eth_news_data.csv'
